In [1]:
import os
import json 
import logging

logging.basicConfig(
    filename='log/app.log',            # Specify the log file name
    level=logging.DEBUG,           # Set the log level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(levelname)s - %(message)s'  # Set the log format
)

# Load the environment configuration JSON data
json_path = 'env_config.json'
with open(json_path, 'r') as file:
    env_config = json.load(file)

hf_home = env_config['HF_HOME']
# Set the HF_HOME environment variable
os.environ['HF_HOME'] = hf_home
# Set the access token to huggingface hub
access_token = env_config['access_token']
os.environ['HUGGINGFACE_HUB_TOKEN'] = access_token

In [2]:
import transformers 
print(transformers.__version__)

from transformers import pipeline
import torch

from accelerate import Accelerator
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from transformers import LlamaTokenizerFast
import torch.nn.functional as F


accelerator = Accelerator()
device = accelerator.device
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-8B"  # non-instruct version

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    # device_map=device,
    token=access_token,
)

tokenizer = LlamaTokenizerFast.from_pretrained(model_id, token=access_token)
tokenizer.pad_token = tokenizer.eos_token

/afs/crc.nd.edu/user/d/dpan/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.44.0


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.61s/it]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [3]:
from llmexp.helper import LlmExpHelper
from datasets import load_dataset
from torch.utils.data import DataLoader

# imdb = load_dataset("imdb")
ds = load_dataset("rajpurkar/squad")
train_ds = ds['train']
test_ds = ds['validation']
# from datasets import load_dataset

# ds = load_dataset("stanfordnlp/sst2")
# train_ds = ds['train']
llm_exp_helper = LlmExpHelper(tokenizer, 'squad')
collate_fn = llm_exp_helper.get_collate_fun()

# Define batch size here!
batch_size = 16
train_dataloader = DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
test_dataloader = DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

In [4]:
# next(iter(train_dataloader))
train_ds[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [5]:
# for name, param in mask_gen_model.named_parameters():
#     print(name, param.device)

In [6]:
from llmexp.squad_model_lora import MaskGeneratingModel
from peft import LoraConfig, get_peft_model

mask_gen_model = MaskGeneratingModel(hidden_size=4096, mlp_hidden_dim=4096, mlp_bottleneck_dim=768, mlp_num_blocks=5)
mask_gen_model.to(device)

target_modules = []
num_layers = 6  # BERT-base 有 12 层
for i in range(num_layers):
    target_modules.extend([
        f"explain_map.layer.{i}.attention.self.query",
        f"explain_map.layer.{i}.attention.self.key",
        f"explain_map.layer.{i}.attention.self.value",
        f"explain_map.layer.{i}.attention.output.dense",
        f"explain_map.layer.{i}.intermediate.dense",
        f"explain_map.layer.{i}.output.dense"
    ])

lora_config = LoraConfig(
    r=4,  # 低秩矩阵的秩
    lora_alpha=32,  # LoRA 的缩放因子
    target_modules= target_modules,  # 目标模块
    lora_dropout=0.1  # Dropout 概率
)
mask_gen_model = get_peft_model(mask_gen_model, lora_config)


from tqdm import tqdm

# Set pad_token_id if it is not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

pad_token_id = tokenizer.pad_token_id

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

optimizer = torch.optim.Adam(mask_gen_model.parameters(), lr=1e-4)

In [7]:
mask_gen_model.train()
for epoch in range(1):
    pbar = tqdm(train_dataloader)
    for idx, data in enumerate(pbar):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        context_mask = data['context_mask'].to(device)
        # get generated texts
        gen_outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=128,
            eos_token_id=terminators,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            return_dict_in_generate=True,
            output_scores=True,
        )
        gen_tokens = gen_outputs.sequences
        pad_length = gen_tokens.size(1) - input_ids.size(1)
        # get the attention mask for the generated tokens, and also mask the padding tokens
        gen_attention_mask = F.pad(attention_mask, (0, pad_length), mode='constant', value=1)
        # (gen_tokens != pad_token_id).long() is the tokens mask, 1 for real tokens and 0 for padding tokens
        unpaded_token_mask = (gen_tokens != pad_token_id).long()
        unpaded_token_mask[:, :-pad_length] = 1
        gen_attention_mask = gen_attention_mask * unpaded_token_mask
        # print(gen_tokens[0])
        # print(gen_attention_mask[0])
        # get the response mask, which is the mask for the generated tokens (the user inputs are masked with 0)
        response_mask = gen_attention_mask.clone()
        response_mask[:, :-pad_length] = 0 # TODO: 有问题. 有问题吗？

        context_mask = F.pad(context_mask, (0, pad_length), mode='constant', value=0)

        # Get the last hidden state for the prompt + response sequence
        with torch.no_grad():
            full_outputs = model(input_ids=gen_tokens, attention_mask=gen_attention_mask, output_hidden_states=True, return_dict=True)
            last_hidden_state = full_outputs.hidden_states[-1]
            last_hidden_state = last_hidden_state.float()
        
        mask_logits = mask_gen_model(last_hidden_state)

        mask_gen_outputs = mask_gen_model.loss_func(model, gen_tokens, gen_attention_mask, context_mask, mask_logits, response_mask, 
                                                                           num_samples=1)
        loss, reward_loss, mask_loss, mask_mean, mean_reward = mask_gen_outputs['loss'], mask_gen_outputs['reward_loss'], mask_gen_outputs['mask_loss'], mask_gen_outputs['mask_mean'], mask_gen_outputs['mean_reward']
        log = (f"Epoch {epoch+1}, Step {idx+1}: Loss = {loss.item():.4f}, " 
                             f"Reward Loss = {reward_loss.item():.4f}, "
                             f"Mean Reward = {mean_reward.mean().item():.4f},"
                             f"advantage = {mask_gen_outputs['advantage'].item():.4f}, "
                             f"Mask_loss = {mask_loss.item():.4f} "
                             f"mask_mean = {mask_mean.item():.4f}"

        )
        pbar.set_description(log)
        logging.debug(log)
    
        # the parameters before updating
        params_before = mask_gen_model.state_dict()

        # Train the model (inner loop)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # # the mask_prob after the updates
        # with torch.no_grad():
        #     mask_logits_after = mask_gen_model(last_hidden_state)

        #     mask_gen_outputs_after = mask_gen_model.loss_func(model, gen_tokens, gen_attention_mask, context_mask, mask_logits_after, response_mask, 
        #                                                                     num_samples=5)
        #     loss_after, reward_loss_after, mask_loss_after, mask_mean_after, mean_reward_after = mask_gen_outputs_after['loss'], mask_gen_outputs_after['reward_loss'], mask_gen_outputs_after['mask_loss'], mask_gen_outputs_after['mask_mean'], mask_gen_outputs_after['mean_reward']
        #     mask_prob_after = (torch.sigmoid(mask_logits_after) * context_mask).clone().detach()
        #     mean_reward_after = mean_reward_after.clone().detach()

        # # load the parameters before the updates
        # mask_gen_model.load_state_dict(params_before)
        # mask_logits_before = mask_gen_model(last_hidden_state)

        # mask_gen_outputs_before = mask_gen_model.loss_func(model, gen_tokens, gen_attention_mask, context_mask, mask_logits_before, response_mask, 
        #                                                                    num_samples=5)
        # loss_before, reward_loss_before, mask_loss_before, mask_mean_before, mean_reward_before = mask_gen_outputs_before['loss'], mask_gen_outputs_before['reward_loss'], mask_gen_outputs_before['mask_loss'], mask_gen_outputs_before['mask_mean'], mask_gen_outputs_before['mean_reward']
        # mask_prob_before = (torch.sigmoid(mask_logits_before) * context_mask)

        # # calculate the ratio of the mask probabilities before and after the updates
        # ratio = mask_prob_after / (mask_prob_before + 1e-6)

        # # 定义PPO的损失函数，假设clip_param是你定义的剪切参数
        # clip_param = 0.2
        # advantage = (mean_reward_after - mean_reward_before).unsqueeze(-1)  # 计算优势函数（advantage），这是根据任务定义的
        # surr1 = ratio * advantage
        # surr2 = torch.clamp(ratio, 1 - clip_param, 1 + clip_param) * advantage
        # ppo_loss = -torch.min(surr1, surr2).mean()

        # # 更新模型参数
        # optimizer.zero_grad()
        # ppo_loss.backward()
        # optimizer.step()

        if idx % 10 == 0:
            print()
        if idx % 100 == 0 and idx != 0:
            torch.save(mask_gen_model.state_dict(), f'saved_model/mask_gen_model_lora_{epoch}_{idx}.pth') 
            print()
            # break

  0%|          | 0/5475 [00:00<?, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
Epoch 1, Step 1: Loss = 0.2222, Reward Loss = 0.2222, Mean Reward = 0.4186,advantage = 0.2186, Mask_loss = 0.0816 mask_mean = 0.4006:   0%|          | 1/5475 [00:03<5:55:26,  3.90s/it]

Epoch 1, Step 11: Loss = 0.2542, Reward Loss = 0.2542, Mean Reward = 0.4989,advantage = 0.2989, Mask_loss = 0.1627 mask_mean = 0.5110:   0%|          | 11/5475 [00:41<5:31:32,  3.64s/it]

Epoch 1, Step 21: Loss = 0.2855, Reward Loss = 0.2855, Mean Reward = 0.7857,advantage = 0.5857, Mask_loss = 0.4769 mask_mean = 0.7666:   0%|          | 21/5475 [01:17<5:02:57,  3.33s/it]

Epoch 1, Step 31: Loss = 0.2497, Reward Loss = 0.2497, Mean Reward = 0.6538,advantage = 0.4538, Mask_loss = 0.3672 mask_mean = 0.7682:   1%|          | 31/5475 [01:54<5:50:41,  3.87s/it]

Epoch 1, Step 41: Loss = 0.3030, Reward Loss = 0.3030, Mean Reward = 0.8009,advantage = 0.6009, Mask_loss = 0.4781 mask_mean = 0.7540:   1%|          | 41/5475 [02:40<6:49:24,  4.52s/it]

...

Epoch 1, Step 211: Loss = 0.1484, Reward Loss = 0.1484, Mean Reward = 0.9934,advantage = 0.7934, Mask_loss = 0.7465 mask_mean = 0.9430:   4%|▍         | 211/5475 [14:03<6:43:17,  4.60s/it]

Epoch 1, Step 221: Loss = 0.1224, Reward Loss = 0.1224, Mean Reward = 1.0030,advantage = 0.8030, Mask_loss = 0.7768 mask_mean = 0.9668:   4%|▍         | 221/5475 [14:44<6:20:56,  4.35s/it]

Epoch 1, Step 231: Loss = 0.1111, Reward Loss = 0.1111, Mean Reward = 0.9820,advantage = 0.7820, Mask_loss = 0.7557 mask_mean = 0.9664:   4%|▍         | 231/5475 [15:23<5:36:32,  3.85s/it]

Epoch 1, Step 234: Loss = 0.0982, Reward Loss = 0.0982, Mean Reward = 1.0322,advantage = 0.8322, Mask_loss = 0.8042 mask_mean = 0.9664:   4%|▍         | 234/5475 [15:39<5:50:35,  4.01s/it]


KeyboardInterrupt: 

In [8]:
import numpy as np
import torch.nn.functional as F

# mask_gen_model.load_state_dict(torch.load('saved_model/mask_gen_model_0_1600.pth',map_location=device))

mask_gen_model.eval()

# tokens = tokenizer.convert_ids_to_tokens(gen_tokens[idx])
# texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
# texts = "I did not like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."
# texts = "I don't like this movie."
# texts = "I really love this film."
# texts = "I really love this film. The acting was great, and the story was amazing. I would recommend this movie to everyone."
# # texts = "I don't like this movie. The acting was terrible, and the story was boring. I would not recommend this movie to anyone."
# messages_lambda = lambda texts: [
#             {"role": "system", "content": "Answer the question based on the context."},
#             # {"role": "system", "content": "You are a chatbot for sentimate analysis."},
#             {"role": "user", "content": texts},
#         ]
# messages = messages_lambda(texts)
# messages_with_template_applied = tokenizer.apply_chat_template(
#             messages,
#             tokenize=False,
#             add_generation_prompt=True,
#         )

# # test_text = [{"text": texts, "label": None}]
# test_text = [{"sentence": texts, "label": None}]
# test_inputs = collate_fn(test_text).to(device)

test_inputs = next(iter(test_dataloader)).to(device)
# test_inputs = next(iter(train_dataloader)).to(device)

# tokens = tokenizer.convert_ids_to_tokens(test_inputs['input_ids'][idx])

# generate the answer for the test inputs
gen_outputs = model.generate(
            input_ids=test_inputs['input_ids'],
            attention_mask=test_inputs['attention_mask'],
            max_new_tokens=128,
            eos_token_id=terminators,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            return_dict_in_generate=True,
            output_scores=True,
        )
input_ids = test_inputs['input_ids']
attention_mask = test_inputs['attention_mask']
gen_tokens = gen_outputs.sequences
pad_length = gen_tokens.size(1) - input_ids.size(1)
# get the attention mask for the generated tokens, and also mask the padding tokens
gen_attention_mask = F.pad(attention_mask, (0, pad_length), mode='constant', value=1)
context_mask = F.pad(test_inputs['context_mask'], (0, pad_length), mode='constant', value=0)
# (gen_tokens != pad_token_id).long() is the tokens mask, 1 for real tokens and 0 for padding tokens
unpaded_token_mask = (gen_tokens != pad_token_id).long()
unpaded_token_mask[:, :-pad_length] = 1
gen_attention_mask = gen_attention_mask * unpaded_token_mask

with torch.no_grad():
    # prompt_outputs = model(input_ids=test_inputs['input_ids'], attention_mask=test_inputs['attention_mask'], output_hidden_states=True, return_dict=True)
    prompt_outputs = model(input_ids=gen_tokens, attention_mask=gen_attention_mask, output_hidden_states=True, return_dict=True)

    last_hidden_state = prompt_outputs.hidden_states[-1].float()
    mask_logits = mask_gen_model(last_hidden_state)




In [9]:
import random
idx = random.randint(0, 4)
test_ids = gen_tokens[idx]
test_mask = gen_attention_mask[idx]
test_mask_prob = torch.sigmoid(mask_logits[idx])
# inverse TODO
# test_mask_prob = 1 - test_mask_prob
test_context_mask = context_mask[idx]

test_tokens = tokenizer.convert_ids_to_tokens(test_ids)
scores = test_mask_prob * test_context_mask
def normalize_except_zeros(array):
    # Create a mask to identify non-zero elements
    mask = array != 0
    
    # Extract non-zero elements
    non_zero_elements = array[mask]
    
    # Normalize non-zero elements
    min_val = np.min(non_zero_elements)
    max_val = np.max(non_zero_elements)
    normalized_non_zero_elements = (non_zero_elements - min_val) / (max_val - min_val)
    
    # Create a copy of the original array to preserve zero values
    normalized_array = np.copy(array)
    
    # Assign normalized values back to the corresponding positions
    normalized_array[mask] = normalized_non_zero_elements
    
    return normalized_array
# scores = normalize_except_zeros(scores.detach().cpu().numpy())

# remove special tokens
filtered_token_scores = [(token, score) for token, score in zip(test_tokens, scores) if token not in tokenizer.all_special_tokens]

# combine subwords
merged_tokens_scores = []
current_token = ""
current_score = 0
count = 0

for token, score in filtered_token_scores:
    if token.startswith("Ġ"):
        if current_token:
            merged_tokens_scores.append((current_token, current_score / count))
            # merged_tokens_scores.append((" ", 0))  # 添加空格
        current_token = token[1:] # remove the speical character
        current_score = score
        count = 1
    elif token.endswith("Ċ"):
        if current_token:
            merged_tokens_scores.append((current_token, current_score / count))
        merged_tokens_scores.append(("<br><br>", 0))  # 添加换行符
        current_token = ""
        current_score = 0
        count = 0
    else:
        current_token += token
        current_score += score
        count += 1

if current_token:
    merged_tokens_scores.append((current_token, current_score / count))


# 根据分数高亮文本（示例中使用HTML标签）
highlighted_text = ""
for token, score in merged_tokens_scores:
    # 动态设置背景颜色：score为0时为白色，score为1时为绿色
    red = int((1 - score) * 255)
    green = 255
    blue = int((1 - score) * 255)
    color = f'rgb({red}, {green}, {blue})'
    highlighted_text += f'<span style="background-color: {color}; color: black;">{token}</span> '

# 打印高亮后的文本
from IPython.display import display, HTML
display(HTML(highlighted_text.strip()))

In [ ]:
merged_tokens_scores

[('<|start_header_id|>system<|end_header_id|>', tensor(0., device='cuda:1')),
 ('<br><br>', 0),
 ('You', tensor(0., device='cuda:1')),
 ('are', tensor(0., device='cuda:1')),
 ('a', tensor(0., device='cuda:1')),
 ('chatbot', tensor(0., device='cuda:1')),
 ('for', tensor(0., device='cuda:1')),
 ('answering', tensor(0., device='cuda:1')),
 ('questions.', tensor(0., device='cuda:1')),
 ('You', tensor(0., device='cuda:1')),
 ('can', tensor(0., device='cuda:1')),
 ('help', tensor(0., device='cuda:1')),
 ('users', tensor(0., device='cuda:1')),
 ('with', tensor(0., device='cuda:1')),
 ('their', tensor(0., device='cuda:1')),
 ('questions', tensor(0., device='cuda:1')),
 ('via', tensor(0., device='cuda:1')),
 ('concise', tensor(0., device='cuda:1')),
 ('responses.<|start_header_id|>user<|end_header_id|>',
  tensor(0., device='cuda:1')),
 ('<br><br>', 0),
 ('Question:', tensor(0.7202, device='cuda:1')),
 ('What', tensor(0.8791, device='cuda:1')),
 ('is', tensor(0.2633, device='cuda:1')),
 ('in', 

In [ ]:
test_mask_prob * test_context_mask

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [ ]:
expl_raw

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [ ]:
mask_prob = torch.sigmoid(mask_logits)
(mask_prob * context_mask).sum(-1) / context_mask.sum(dim=-1)

tensor([0.1241, 0.2500, 0.7787, 0.0909, 0.1908, 0.1361, 0.3316, 0.5104, 0.2696,
        0.1001, 0.5931, 0.0812, 0.4139, 0.3334, 0.4853, 0.2987],
       device='cuda:0')

In [ ]:
mask_prob

tensor([[9.9998e-01, 1.3412e-02, 2.1168e-05, 7.2758e-04, 9.5831e-01, 2.8712e-06,
         1.9530e-03, 6.8251e-04, 9.6814e-01, 7.6582e-04, 2.2164e-06, 1.7009e-02,
         2.4596e-09, 1.4519e-04, 3.0463e-06, 7.9092e-04, 4.0336e-06, 5.2874e-05,
         1.4755e-07, 5.3417e-09, 1.0859e-07, 8.2496e-07, 1.0534e-05, 2.1604e-06,
         6.6980e-10, 1.8266e-01, 5.4030e-07, 4.0081e-02, 3.5185e-04, 3.1933e-01,
         3.2836e-09, 1.5197e-06, 8.2054e-01, 8.0839e-01, 5.4567e-01, 4.8829e-01,
         1.0000e+00, 4.3639e-05, 3.5097e-07, 3.8482e-09, 1.9672e-04, 1.1242e-07,
         2.7982e-13, 5.4683e-09, 4.5730e-01, 9.5505e-01, 6.7815e-04, 7.4368e-02,
         9.9924e-01, 6.6332e-03, 1.0508e-08, 1.8774e-01]], device='cuda:0')

In [ ]:
test_inputs['input_ids'][idx]

tensor([128000, 128006,   9125, 128007,    271,   2675,    527,    264,   6369,
          6465,    369,  27065,   6492,     13,   1472,    649,   1520,   3932,
           449,    872,   4860,   4669,  64694,  14847,    315,  27592,  45450,
            11,    477,  85165,  24093,     13, 128009, 128006,    882, 128007,
           271,   2028,   5818,    574,    279,   1888,   5818,    358,    617,
          3596,   3970,      0,   1063,  16451,   1051,  27873,     11,    719,
         15718,    574,   2294,     13, 128009, 128006,  78191, 128007,    271],
       device='cuda:0')

In [ ]:
tokenizer.convert_ids_to_tokens(271)

'ĊĊ'

[0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.6432 0.8566 0.5179
 0.2417 0.     0.1211 0.3355 0.618  0.5345 0.1401 0.3401 0.4729 0.3531
 0.661  0.7049 0.0297 0.1724 0.9905 1.     0.1606 0.1107 0.2363 0.2891
 0.116  0.0777 0.     0.     0.     0.     0.     0.     0.     0.    ]


In [ ]:
(torch.sigmoid(mask_logits) * context_mask)[idx]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.8527, 0.1770, 0.1719, 0.1660, 0.1613, 0.1549, 0.1581, 0.1622, 0.1666,
        0.1635, 0.1568, 0.1589, 0.1618, 0.1634, 0.1668, 0.1647, 0.1546, 0.1576,
        0.1788, 0.1779, 0.1592, 0.1581, 0.1607, 0.1619, 0.1586, 0.1583, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
       device='cuda:0')

In [ ]:
torch.sigmoid(mask_logits)

tensor([[0.4822, 0.4821, 0.4816,  ..., 0.4920, 0.4874, 0.4901],
        [0.4843, 0.4851, 0.4855,  ..., 0.4941, 0.4861, 0.4891],
        [0.4785, 0.4805, 0.4805,  ..., 0.4918, 0.4823, 0.4864],
        ...,
        [0.4753, 0.4758, 0.4761,  ..., 0.4847, 0.4761, 0.4802],
        [0.4876, 0.4883, 0.4882,  ..., 0.4887, 0.4880, 0.4943],
        [0.4843, 0.4851, 0.4852,  ..., 0.4946, 0.4853, 0.4947]],
       device='cuda:0')

In [ ]:
mask_gen_model

MaskGeneratingModel(
  (explain_map): MLP(
    (input_layer): Linear(in_features=4096, out_features=1024, bias=True)
    (attention_layers): ModuleList(
      (0-1): 2 x MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
      )
    )
    (layers): ModuleList(
      (0-1): 2 x Sequential(
        (0): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (1): PReLU(num_parameters=1)
        (2): Linear(in_features=1024, out_features=1024, bias=True)
      )
    )
    (output_layer): Linear(in_features=1024, out_features=1, bias=True)
  )
)

In [ ]:
print(tokens[35])
print(expl[35])

<|end_header_id|>
0.0


In [ ]:
device

device(type='cuda')

In [ ]:
texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
# texts = "I really didn't like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."
# texts = "I don't like this movie."
# texts = "I really love this film."
messages_lambda = lambda texts: [
    {"role": "system", "content": "You are a chatbot for sentimate analysis. You can help users with their questions via concise responses of POSITIVE, or NEGATIVE."},
    # {"role": "system", "content": "You are a chatbot for sentimate analysis."},
    {"role": "user", "content": texts},
]
messages = messages_lambda(texts)
messages_with_template_applied = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
print(messages_with_template_applied)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a chatbot for sentimate analysis. You can help users with their questions via concise responses of POSITIVE, or NEGATIVE.<|eot_id|><|start_header_id|>user<|end_header_id|>

This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


